# 1. Installing dependencies 


In [1]:
# !pip install -r requirements.txt
# !pip install --upgrade numpy
# !pip install --upgrade h5py


# 2a. Model training 
The program applies Transfer Learning to this existing model and re-trains it to classify a new set of images.

This example shows how to take a Inception v3 architecture model trained on ImageNet images,
and train a new top layer that can recognize other classes of images.

You can replace the image_dir argument with any folder containing subfolders of
images. The label for each image is taken from the name of the subfolder it's in.



In [2]:
#TEMPORARY (removing datafiles from dataset to reduce amount of files)
# import os
# import random

# # Set the path to your dataset
# dataset_path = './dataset'

# # Set the number of images you want to delete from each class
# num_images_to_delete = 2500  # Change this to the number you want

# # Iterate over each class directory
# for class_dir in os.listdir(dataset_path):
#     class_dir_path = os.path.join(dataset_path, class_dir)

#     # Check if it's a directory
#     if os.path.isdir(class_dir_path):
#         # Get all images in the directory
#         images = [os.path.join(class_dir_path, img) for img in os.listdir(class_dir_path)]
#         # Shuffle the list to get random images to delete
#         random.shuffle(images)

#         # Delete the specified number of images
#         for img_to_delete in images[:num_images_to_delete]:
#             os.remove(img_to_delete)
#             print(f"Deleted {img_to_delete}")

# print("Deletion complete.")


In [6]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Assuming there are 29 classes based on the folders A-Z, del, nothing, space
your_number_of_classes = 29
path_to_your_train_data = './dataset'  # Update this path to where your dataset is located

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False)

# Freeze the base model
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # New FC layer, random init
predictions = Dense(your_number_of_classes, activation='softmax')(x)  # New softmax layer
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    path_to_your_train_data,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Train the model
model.fit(train_generator, steps_per_epoch=train_generator.n // 32, epochs=10)

# Optionally, fine-tune the model by unfreezing layers and recompiling the model


Found 85428 images belonging to 29 classes.
Epoch 1/10
2669/2669 [==============================] - 1847s 691ms/step - loss: 0.6323 - accuracy: 0.8324
Epoch 2/10
  45/2669 [..............................] - ETA: 20:36 - loss: 0.2851 - accuracy: 0.9194

KeyboardInterrupt: 

# 2b. Fine-tuning the model

In [7]:
# Unfreeze the base model
base_model.trainable = True

# It's important to freeze all layers up to the ones you want to fine-tune
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),  # Note the lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model
model.fit(train_generator, 
          steps_per_epoch=train_generator.n // 32, 
          epochs=5)  # You may not need as many epochs as before


Epoch 1/5
   9/2669 [..............................] - ETA: 22:43 - loss: 0.9945 - accuracy: 0.6736

KeyboardInterrupt: 

# 3. Classification / Model Testing

In [9]:
# import os

# def classify_folder(folder_path, model, generator):
#     for filename in os.listdir(folder_path):
#         if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#             image_path = os.path.join(folder_path, filename)
#             img_array = prepare_image(image_path)
#             predictions = model.predict(img_array)
#             class_index = np.argmax(predictions[0])
#             class_label = get_class_label(class_index, generator)
#             probability = np.max(predictions[0])
#             print(f"Image: {filename} - Prediction: {class_label} (Score = {probability:.5f})")

# # Path to your folder containing images
# folder_path = './testimages'  # Update this to your folder path

# # Classify all images in the folder
# classify_folder(folder_path, model, train_generator)


1/1 [==============================] - 0s 48ms/step
Image: testimage1 -d.jpg - Prediction: D (Score = 0.99753)
1/1 [==============================] - 0s 33ms/step
Image: testimage2 - del.jpg - Prediction: del (Score = 0.69086)


In [8]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

def prepare_image(file_path):
    img = image.load_img(file_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)

def get_class_label(class_index, generator):
    # Assuming generator.class_indices is a dict mapping class names to class indices
    class_labels = {v: k for k, v in generator.class_indices.items()}
    return class_labels[class_index]

def classify_folder(folder_path, model, generator):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            img_array = prepare_image(image_path)
            predictions = model.predict(img_array)
            class_index = np.argmax(predictions[0])
            class_label = get_class_label(class_index, generator)
            probability = np.max(predictions[0])
            print(f"Image: {filename} - Prediction: {class_label} (Score = {probability:.5f})")

# Assuming 'model' and 'train_generator' are defined as per your previous setup
# Path to your folder containing images
folder_path = './testimages'  # Update this to your folder path

# Classify all images in the folder
classify_folder(folder_path, model, train_generator)

1/1 [==============================] - 1s 610ms/step
Image: testimage1 -d.jpg - Prediction: D (Score = 0.99753)
1/1 [==============================] - 0s 37ms/step
Image: testimage2 - del.jpg - Prediction: del (Score = 0.69086)


# 3b. Classification with Webcam

In [10]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [13]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# Load your trained model
model.save('model.h5')
model = load_model('model.h5')

# Define a function to preprocess the frame captured by the webcam
def prepare_frame(frame):
    # Convert the frame to RGB, resize, convert to array, add a batch dimension, and preprocess
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (224, 224))
    frame_array = img_to_array(frame_resized)
    frame_expanded = np.expand_dims(frame_array, axis=0)
    return preprocess_input(frame_expanded)

# Initialize webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Read and classify frames in real-time
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break  # If frame is not captured successfully, exit loop

    # Preprocess the captured frame
    preprocessed_frame = prepare_frame(frame)
    
    # Make predictions on the preprocessed frame
    predictions = model.predict(preprocessed_frame)
    class_index = np.argmax(predictions, axis=1)[0]
    probability = np.max(predictions)
    
    # Optionally, retrieve the class label using your get_class_label function
    # class_label = get_class_label(class_index, train_generator)  # Assuming train_generator is accessible

    # Display the classification result on the frame
    cv2.putText(frame, f'Class: {class_index}, Prob: {probability:.2f}', (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Webcam Feed - Press Q to Exit', frame)

    # Break the loop when 'Q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step


# 4. Model Evaluation

In [14]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Assuming 'model' is your trained model and 'test_dir' is the path to your test dataset

# Load and preprocess test dataset
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    folder_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)  # Important: keep data in same order as labels

# Make predictions
predictions = model.predict(test_generator, steps=np.ceil(test_generator.samples/test_generator.batch_size))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Compute Precision, Recall, F1 score
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Compute Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
print(cm)

# Compute ROC-AUC for multi-class by treating it as One-vs-All
roc_auc = roc_auc_score(true_classes, predictions, multi_class='ovr')
print(f'ROC-AUC: {roc_auc}')

# Plot ROC Curves for each class
fpr = {}
tpr = {}
thresh ={}
roc_auc_dict = {}
n_class = len(class_labels)

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(true_classes, predictions[:,i], pos_label=i)
    roc_auc_dict[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], linestyle='--', label=f'Class {class_labels[i]} vs Rest (area = {roc_auc_dict[i]:.2f})')

plt.title('Multi-class ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='best')
plt.show()


Found 0 images belonging to 0 classes.


ValueError: Asked to retrieve element 0, but the Sequence has length 0